In [5]:
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.optimize import curve_fit
import math as m

In [6]:
#os.listdir()

In [10]:
filename = "11_01_19-3.scan"

f = open(filename)
no_of_mass = 0
start = False
raw_datas = []
iterations = []
for line in f:
    
    if line.find("#mass")>=0:
        no_of_mass += 1
        tmp = line.split(":")[-1].strip()
        iterations.append(int(tmp))
        
    if not line[0] == "#" and not line == "\n":
        if line.strip() == "ALL:":
            start = True
        if start:
            raw_datas.append(line)
del raw_datas[0]

f.close()

all_datas = [[float(j) for j in i.split()] for i in raw_datas]
data_run_cycle = int(len(all_datas)/sum(iterations))

mass = [[] for i in range(no_of_mass)]
k =0
m = 0
for i in iterations:
    j = i*data_run_cycle
    mass[m] = all_datas[k:k+j]
    k += j
    m += 1
    
mass_values = [i[0][0] for i in mass]

In [23]:
len(mass), len(iterations), data_run_cycle, mass_values

(5, 5, 16, [88.88, 88.88, 98.88, 108.88, 118.88])

In [105]:
mass

[[[88.88, 50.0, 836.0],
  [88.88, 513.333, 1168.0],
  [88.88, 976.667, 2011.0],
  [88.88, 1440.0, 2436.0],
  [88.88, 1903.333, 2874.0],
  [88.88, 2366.667, 3791.0],
  [88.88, 2830.0, 3733.0],
  [88.88, 3293.333, 4406.0],
  [88.88, 3756.667, 4214.0],
  [88.88, 4220.0, 4224.0],
  [88.88, 4683.333, 4120.0],
  [88.88, 5146.667, 4282.0],
  [88.88, 5610.0, 4566.0],
  [88.88, 6073.333, 4738.0],
  [88.88, 6536.667, 4749.0],
  [88.88, 7000.0, 5127.0]],
 [[88.88, 50.0, 886.0],
  [88.88, 513.333, 1300.0],
  [88.88, 976.667, 1752.0],
  [88.88, 1440.0, 2670.0],
  [88.88, 1903.333, 2922.0],
  [88.88, 2366.667, 3206.0],
  [88.88, 2830.0, 3771.0],
  [88.88, 3293.333, 4125.0],
  [88.88, 3756.667, 3690.0],
  [88.88, 4220.0, 4664.0],
  [88.88, 4683.333, 4306.0],
  [88.88, 5146.667, 5548.0],
  [88.88, 5610.0, 5521.0],
  [88.88, 6073.333, 4736.0],
  [88.88, 6536.667, 4910.0],
  [88.88, 7000.0, 5723.0],
  [88.88, 50.0, 764.0],
  [88.88, 513.333, 1024.0],
  [88.88, 976.667, 2134.0],
  [88.88, 1440.0, 2413.0]